In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install emoji

     |████████████████████████████████| 325 kB 11.9 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 136 kB 59.8 MB/s 
     |████████████████████████████████| 212 kB 47.8 MB/s 
     |████████████████████████████████| 1.1 MB 44.8 MB/s 
     |████████████████████████████████| 127 kB 69.2 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 59.5 MB/s 
     |████████████████████████████████| 144 kB 38.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 11.2 MB/s

In [ ]:
import os
import random
import numpy as np
import glob
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
import json
import os
from datasets import Dataset
from transformers import *
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import re
import gc
import emoji
import zipfile

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
batch_size = 512
max_length = 128
start_lr = 1e-4
min_lr = 1e-8
epochs = 10
seed = 42

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    #map to 01
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    roc = roc_auc_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
model_ckpt = "distilbert-base-uncased"
# tokenizer = DistilBertTokenizerFast.from_pretrained(model_ckpt)

In [ ]:
input_ids = np.load('/content/drive/MyDrive/datasocio/input_ids_s.npy', allow_pickle = True)
attention_masks= np.load('/content/drive/MyDrive/datasocio/attention_masks_s.npy', allow_pickle = True)
target = np.load('/content/drive/MyDrive/datasocio/target_s.npy',allow_pickle = True)
# target = np.array(pd.get_dummies(X['labels']))

In [ ]:
config = AutoConfig.from_pretrained(model_ckpt) 
backbone = TFDistilBertModel.from_pretrained(model_ckpt, config = config)

In [ ]:
backbone.summary()

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
trainloader = tf.data.Dataset.from_tensor_slices(({'input_ids':input_ids, 'attention_mask':attention_masks}))
trainloader = (
        trainloader
        .batch(batch_size)
        .prefetch(AUTOTUNE)     
    )

In [ ]:
len(trainloader)

In [ ]:
#use tqdm next time
i = 0
for batch in tqdm(iter(trainloader), total = len(trainloader)):
  embeddings = backbone(batch)[0].numpy()
  np.save(f'embeddings_dbert{i}.npy', embeddings, allow_pickle = True)
  i += 1
  del embeddings
  gc.collect()

In [ ]:
np.save('/content/drive/MyDrive/datasocio/embeddings_dbert2.npy', predictions, allow_pickle = True)

In [ ]:
np.append(predictions, embeddings).shape

In [ ]:


def build_model():
    
    input_ids = tf.keras.Input(shape=(128,),dtype='int32', name = 'input_ids')
    attention_masks = tf.keras.Input(shape=(128,),dtype='int32', name = 'attention_mask')
    
    output = backbone(input_ids,attention_masks)[0]

    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    
    
    for layer in model.layers[:3]:
        print(layer)
        layer.trainable = False
    
    return model

In [ ]:
features = build_model()
features.compile()

In [ ]:
for batch in iter(trainloader):
  embeddings = features.predict(batch, batch_size = batch_size)
  predictions.append(embeddings)
  break

In [ ]:
embeddings.shape

In [ ]:
embeddings[0].shape

In [ ]:
backbone(batch)